### 학습 목표

- 초대용량 데이터는 어떻게 처리할까요?


- 데이터를 분할하여 불러오고 처리하는 방법에 대해서 연습해봅니다.


- 불량 검출 문제에 대해 살펴봅니다.


- Class imbalance 문제를 해결하는 방법에 대해 알아봅니다.

## Bosch Production Line Performance

- 공정과정 데이터를 통해 제품 내부 불량 검출하기

- 주어진 공정변수를 이용하여 정상/불량 분류하기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train_numeric_part = pd.read_csv('../input/bosch-production-line-performance/train_numeric.csv.zip',
                                nrows=1000)# 데이터를 불러올 때 몇개의 Row를 불러올것인가
#                                 usecols=,# 불러올 column들을 지정
#                                 chunksize=,) # 데이터를 매번 불러올 때 몇개의 row를 불러올것인가
train_categorical_part = pd.read_csv('../input/bosch-production-line-performance/train_categorical.csv.zip',
                                nrows=1000)# 데이터를 불러올 때 몇개의 Row를 불러올것인가
train_date_part = pd.read_csv('../input/bosch-production-line-performance/train_date.csv.zip',
                                nrows=1000)# 데이터를 불러올 때 몇개의 Row를 불러올것인가
print(train_numeric_part.shape, train_categorical_part.shape, train_date_part.shape)

In [ ]:
train_numeric_part
train_categorical_part
train_date_part

### 1. 대용량 데이터 다루기

**Technique1. 잘라서 불러오기 (read csv 함수를 사용하는 방법)**

1) row 단위로 일부를 가져오기
- nrows : 지정한 row 개수만큼만 잘라서 가져옵니다. (위에서부터)
- chunksize : for문을 사용해서 매번 특정 row개수씩 가져옵니다.(위에서부터 순서대로)

2) column 단위로 일부를 가져오기
- usecos : 사용할 column들만 지정해서 가져옵니다.


**Technique2. Reducing Memory Technique (dtype 변경하기)**

In [ ]:
# for chunk in pd.read_csv('../input/bosch-production-line-performance/train_date.csv.zip',chunksize=100000,usecols=S0_cols):
#     print(chunk.sum())

In [ ]:
# Technique 1.
## 1) train_date_part를 가지고, 각 station별 feature를 뽑아봅니다.
### 가정. station별로 측정 시간이 모두 같으면, 하나씩만 사용하자. (X)
### 결론. station별 측정 시간을 평균값으로 사용하자 (time feature)
station_names=train_date_part.count().reset_index()['index'].str.split('_',expand=True)[1][1:].drop_duplicates().values
train_date=pd.DataFrame()
chunks=pd.read_csv('../input/bosch-production-line-performance/train_date.csv.zip',chunksize=100000)
for chunk in chunks:
    temp_date =pd.DataFrame()

    for station in station_names : 
        station_cols = chunk.columns[chunk.columns.str.contains(station)]
        col_means = chunk[station_cols].mean(axis=1) #각 station에 대해서 row별 평균
        temp_date[station] = col_means
    #     print(col_means)
    train_date = pd.concat([train_date,temp_date]) # 각 station별 측정시간의 평균으로 데이터를 요약한 정보.
train_date

In [ ]:
train_date.info()

In [ ]:
a=train_numeric_part.isnull().sum() #column별 결측치
b=train_numeric_part.isnull().sum() #column별 결측치
# len(train_numeric_part)
a+b

In [ ]:
## TO-DO 모범답안
# train_numeric 데이터를 불러와서, 전체 결측치 비율을 column별로 측정한 다음에
# 결측치가 50%를 넘는 모든 column을 제거합니다.

chunks=pd.read_csv('../input/bosch-production-line-performance/train_numeric.csv.zip',chunksize=100000)

missing_ratio = pd.Series(index=train_numeric_part.columns,
                         data=np.zeros(len(train_numeric_part.columns)))
n_data=0

for chunk in chunks:
    temp = chunk.isnull().sum()
    n_data=n_data + len(chunk)
    
    missing_ratio = missing_ratio +temp
    
missing_ratio = missing_ratio / n_data # -> 비율로 변경
missing_ratio

## TO-DO
# train_numeric 데이터를 불러와서, 전체 결측치 비율을 column별로 측정한 다음에
# 결측치가 50%를 넘는 모든 column을 제거합니다.

# check_numeric=pd.DataFrame()
# chunks=pd.read_csv('../input/bosch-production-line-performance/train_numeric.csv.zip',chunksize=100000)
# total=0
# for chunk in chunks:
#     temp_numeric =pd.DataFrame()
    
#     total+=len(chunk)
#     if check_numeric.empty:
#         check_numeric= chunk.isnull().sum()
#     else:
#         check_numeric = check_numeric+chunk.isnull().sum()
# check_numeric/total

In [ ]:
# del_cols=check_numeric[(check_numeric/total*100)>50]
# del_cols.index
drop_cols = train_numeric_part.columns[missing_ratio>=0.5]
usecols = np.setdiff1d(train_numeric_part.columns , drop_cols)

train_numeric=pd.read_csv('../input/bosch-production-line-performance/train_numeric.csv.zip',usecols=usecols)
train_numeric

In [ ]:
# JOIN 연산을 할 기준인 Id column을 추가
train_date.columns = 't_'+ train_date.columns # 시간 관련 feature임을 알리기 위해 column name 변경
train_date["Id"]= train_numeric.Id
train_date

In [ ]:
train = pd.merge(train_numeric, train_date, on="Id") ##inner join
train

In [ ]:
import gc
del train_numeric_part
del train_numeric
del train_date
del train_categorical_part
del train_date_part
gc.collect() # 메모리 청소기

In [ ]:
# 결측치가 50%가 넘는 column들 제거.
# drop_cols= train[train.isnull().mean()>=0.5]
# use_cols= train[train.isnull().mean()<0.5]
# train= train[use_cols]
use_cols=train.columns[train.isnull().mean()<0.5]
train=train[use_cols]
train

In [ ]:
# 결측치를 모두 0으로 채웁니다.
train = train.fillna(0)
train

In [ ]:
# Technique 2.
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [ ]:
train, _ = reduce_mem_usage(train)

### 2. 불량 검출 문제 다루기 (Class Imbalance problem)

Technique 1. Sampling Method (Under / Over)

- 1000000(정상) : 100(불량) = 10000 : 1 ---> 컴퓨터는 모든 데이터를 정상이라고 예층하려고 합니다.
--> 1;1 비율로 변경이 필요합니다! (예측성능이 좋아짐)

**개수가 많은 클래스=magority class, 개수가 적은 클랙스 = minority class**

1) Undersampling
- majority class에서 sampling을 하여, minority class의 수로 맞춰주는 기법.
eg. 1000000 -> 100 -------------> 100: 100: = 1:1
2) Oversamplig
- minority class에서 sampling하여, majority class의 수를 맞춰주는 기법.
eg. 100 -> 10000000 ------> 1000000 : 1000000 = 1:1
3) Hybrid approach (under + over)
- undersampling과 oversapling을 사용하여 1:1의 비율을 맞춰주는 기법.
eg. 1000000 -> 10000, 100 ->10000 ---------> 10000 : 10000 = 1:1



Technique 2. Change to Outlier Detection problem

- DBSCAN, IsolationForest (Unsupervised learning approach): 비지도 학습

semi-supervised learning anomaly detection 

In [ ]:
# Technique 1.
sns.countplot(data=train, x="Response")
train.Response.value_counts(normalize=True)


In [ ]:
normal = train[train.Response ==0]
abnormal = train[train.Response ==1]
#Undersampling
under_normal = normal.sample(n=len(abnormal),random_state=42)
u_train = pd.concat([under_normal,abnormal])
u_train #13758 = 6879 *2



#Hybrid

In [ ]:
#Oversampling = Generation , 원본 데이터의 분포를 학습해서 데이터 생성
from imblearn.over_sampling import SMOTE
X_train = train.drop(columns=["Id","Response"])
y_train = train.Response

# fit_resample 함수가 자동으로 y_train을 보고 minority class를 찾아서 1:1 비율을 맞춰준다
oX_train,oy_train = SMOTE().fit_resample(X_train,y_train)
print(oX_train.shape, oy_train.shape) # (1176868,168) /(1176868,168)

In [ ]:
from imblearn.over_sampling import SMOTE

# Hybrid (under + over)
n_sample = 10000

normal = train[train.Response ==0].sample(n=n_sample,random_state=42)
abnormal = train[train.Response ==1]

temp_df = pd.concat([normal,abnormal]) # 10000 + 6879

X_train = temp_df.drop(columns=["Id","Response"])
y_train = temp_df.Response

# fit_resample 함수가 자동으로 y_train을 보고 minority class를 찾아서 1:1 비율을 맞춰준다
hX_train,hy_train = SMOTE().fit_resample(X_train,y_train)
print(hX_train.shape, hy_train.shape) # (20000,168) /(20000,168)

In [ ]:
hy_train.value_counts()

In [ ]:
# Technique 2.